In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

# stuff for this notebook to work in kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import time

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.debugging.set_log_device_placement(True)
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
path_to_file = './bullets.txt'
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Take a look at the first 250 characters in text
print(text[:250])

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- MAPS TO ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

Length of text: 4461059 characters
112 unique characters
- Pivotal player in 2009 Pumpkin Patrol; ensured safety of kids in base housing while trick-or-treating--promote!
- CAT exec for Air show; enabled coord btwn Wg and civ agencies--104K attendees awed by air power display
- Coordinated six sports activ
{
  &#39;\t&#39;:   0,
  &#39;\n&#39;:   1,
  &#39; &#39; :   2,
  &#39;!&#39; :   3,
  &#39;&quot;&#39; :   4,
  &#39;#&#39; :   5,
  &#39;$&#39; :   6,
  &#39;%&#39; :   7,
  &#39;&amp;&#39; :   8,
  &quot;&#39;&quot; :   9,
  &#39;(&#39; :  10,
  &#39;)&#39; :  11,
  &#39;*&#39; :  12,
  &#39;+&#39; :  13,
  &#39;,&#39; :  14,
  &#39;-&#39; :  15,
  &#39;.&#39; :  16,
  &#39;/&#39; :  17,
  &#39;0&#39; :  18,
  &#39;1&#39; :  19,
  ...
}
&#39;- Pivotal pla&#39; ---- MAPS TO ---- &gt; [15  2 50 75 88 81 86 67 78  2 82 78 67]


In [5]:
# The maximum length sentence we want for a single input in characters
seq_length = 250

examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets. 
# char_dataset is one basically one long 1d array with every element in there.
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNextSync in device /job:localhost/replica:0/task:0/device:CPU:0
-
 
P
i
v
o
t
a
l
 
Executing op DeleteIterator in device /job:localhost/replica:0/task:0/device:CPU:0


In [6]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNextSync in device /job:localhost/replica:0/task:0/device:CPU:0
&#39;- Pivotal player in 2009 Pumpkin Patrol; ensured safety of kids in base housing while trick-or-treating--promote!\n- CAT exec for Air show; enabled coord btwn Wg and civ agencies--104K attendees awed by air power display\n- Coordinated six sports activi&#39;
&#39;ties at Gp picnic; boosted morale/fostered camaraderie of 500--promote soonest!\n- Maintained 43 servers; applied 75 vital patches; protected $322K hardware/$97K in software--99% up-time\n- Coordinated migration of C2 Remedy de

In [7]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [8]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNextSync in device /job:localhost/replica:0/task:0/device:CPU:0
Input data:  &#39;- Pivotal player in 2009 Pumpkin Patrol; ensured safety of kids in base housing while trick-or-treating--promote!\n- CAT exec for Air show; enabled coord btwn Wg and civ agencies--104K attendees awed by air power display\n- Coordinated six sports activ&#39;
Target data: &#39; Pivotal player in 2009 Pumpkin Patrol; ensured safety of kids in base housing while trick-or-treating--promote!\n- CAT exec for Air show; enabled coord btwn Wg and civ agencies--104K attendees awed by air power display\n- Coordinated six sports activi&#39;


In [9]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
Step    0
  input: 15 (&#39;-&#39;)
  expected output: 2 (&#39; &#39;)
Step    1
  input: 2 (&#39; &#39;)
  expected output: 50 (&#39;P&#39;)
Step    2
  input: 50 (&#39;P&#39;)
  expected output: 75 (&#39;i&#39;)
Step    3
  input: 75 (&#39;i&#39;)
  expected output: 88 (&#39;v&#39;)
Step    4
  input: 88 (&#39;v&#39;)
  expected output: 81 (&#39;o&#39;)


In [10]:
# Batch size - the number of simultaneous samples to evaluate on each training step. 
#  So for a given model step, BATCH_SIZE number examples are run through the model at that step, and the results for those 
#  BATCH_SIZE examples are compared to their respective "correct answers," and the resulting averaged or weighted averaged delta
#  is used to adjust the model for the next training set. 
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

Executing op AnonymousRandomSeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0


&lt;BatchDataset shapes: ((128, 250), (128, 250)), types: (tf.int32, tf.int32)&gt;

In [11]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 1024

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
    tf.keras.layers.LSTM(rnn_units,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
    tf.keras.layers.LSTM(rnn_units,
                              return_sequences=True,
                              stateful=True,
                              recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [13]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:

In [14]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Executing op TakeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op IteratorGetNextSync in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ResourceGather in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Transpose in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Split in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ZerosLike in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ConcatV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Split in devic

In [15]:
model.summary()

Model: &quot;sequential&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 512)          57344     
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         6295552   
_________________________________________________________________
lstm_1 (LSTM)                (128, None, 1024)         8392704   
_________________________________________________________________
lstm_2 (LSTM)                (128, None, 1024)         8392704   
_________________________________________________________________
dense (Dense)                (128, None, 112)          114800    
Total params: 23,253,104
Trainable params: 23,253,104
Non-trainable params: 0
_________________________________________________________________


In [16]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

Executing op Multinomial in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Squeeze in device /job:localhost/replica:0/task:0/device:GPU:0


array([109,  61,  84,  98, 102,  85, 105,  56,  26, 108,  70,  69,  64,
        74,  32,  17,   9,  69,  29,  29,  84,  62,  33,   8,  29,  52,
        42,  32,  65,  74,  48,  56, 103,  19,  25,  71,  91,  77,  11,
         3, 111,  23,  90,  57,  43, 110,  77,  53, 106, 105,  23,  73,
        26,  19,  33,  99,  65,  15,  40,  65,  81,  53, 108,  15,  52,
        42,  23,  60,  10,  98,  87,  57,  20,  92,  54,  64,  93, 104,
        68,  24,  43,  14,  19,  46,  52,  75,  89,  86,  34,  58,  37,
        38,  74, 110,  20, 102,  15,   3, 109,  29, 105,  14,  56,  75,
        45,  71,  49, 104,  63,  13,  61,  56,  67,   9,  62,   1,  40,
        66,  65, 102,  49,  54, 110,  13,  73,  79,  46,  24,  23,  86,
        50,  49,  50,  61,  19,  34,  70,  51, 109,  91,  59,  80,  61,
        61,  56,  94,  57, 108, 108,  29,  20,  21,   7,  69,  29,  91,
        61, 109,  41,  80,   2,  75,  20, 102, 111,  62,   0,  16,  57,
        24, 108,   9,  11,  34,  36,  32,  40,  98,  18,  52,  2

In [17]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 &#39;ent--completed 15 semester hrs; 3.8 GPA\n- Volunteered as a designated driver for squadron Christmas party--provided safe transportation for 16 members\n- Completed 15 credit hours towards two separate degrees; maintained 4.0 GPA--six credit hours shy &#39;

Next Char Predictions: 
 &quot;•[r³\u2009s‘V8”dc^h&gt;/&#39;c;;r\\?&amp;;RH&gt;_hNV–17eyk)!\U00100bd75xWI…kS’‘5g81?é_-F_oS”-RH5Z(³uW2zT^{—b6I,1LRiwt@XCDh…2\u2009-!•;‘,ViKeO—]+[Va&#39;\\\nF`_\u2009OT…+gmL65tPOP[1@dQ•yYn[[V}W””;23%c;y[•Gn i2\u2009\U00100bd7\\\t.W6”&#39;)@B&gt;F³0R8)’E=z–en@OZ-zdyw6}UlW$=$H&amp;TaTUbuY}²2Jm aéOS,Ov”(U(t,\u2009)\U00100bd7up$[2X³C,@&#39;=y…&quot;


In [18]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Pack in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Reshape in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op SparseSoftmaxCrossEntropyWithLogits in device /job:localhost/replica:0/task:0/device:GPU:0
Prediction shape:  (128, 250, 112)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.7182508


In [19]:
model.compile(optimizer='adam', loss=loss)

In [20]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [21]:
EPOCHS=75
# for previous model, around it converged to .23 at around 75 epochs. went all the way to 90 and it only dropped to like .21 or so.

In [22]:
with tf.device('/GPU:0'):   # GPU:/1 uses my GTX 970, it's the opposite of what is listed in task manager. With two LSTM layers, 970 couldn't handle I think.
    history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Epoch 1/75
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleO

In [23]:
tf.train.latest_checkpoint(checkpoint_dir)

&#39;./training_checkpoints\\ckpt_75&#39;

In [24]:
model.save('bullets-lstm.h5')

In [25]:
# For the purposes of evaluating the model, setting the batch size to 1 so you can train one thing at a time.

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RestoreV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RestoreV2 in device /job:localhost/replica:0/task:0/device:CPU:0


In [26]:
model.summary()

Model: &quot;sequential_1&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 512)            57344     
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 1024)           6295552   
_________________________________________________________________
lstm_4 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_5 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 112)            114800    
Total params: 23,253,104
Trainable params: 23,253,104
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.save('bullets-lstm_built.h5')

In [28]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.5

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      #print(input_eval)
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [29]:
print(generate_text(model, start_string=u"- "))

Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Squeeze in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:GPU:0
- 520 FW base community minded; raised $300 for hos morale--enrich&#39;d community!
- Led Wg flt mgt OI rewrite edu to msn accomplishment
- Expedited handling of 196 components--maintained hard down time to their zero write-ups
- Created Mx actions during TDY; Id&#39;d/serviced damaged location cables--enabled Nav/TAW-SNot consured response capabilities by 30%
- Executed 80 entire lay inspections; corrected/closed 20 trends...rdy&#39;d Ex RIP HAZMAT Daedalian Manager
- Functioned as exceptional leadership qualities on mission down status--crucial to a taled increased motivated community ties
- Demonstrated leadership &amp; mentorship inprocessing; challenge w/inclusive flt subs..

In [30]:
print(generate_text(model, start_string=u"- Led "))


- Led 4 trucks; coord&#39;d mvmt f/17 prsnl/22 parts--propelled 316 muns/1.4K hrs/3K sorties
- Deployed to Ex RILIKE support for 5 nt promote board
- Directs section&#39;s most current inspections; ID&#39;d potential liability--receipts submitted/trng conducted/2 pilot units by Sq/CC
- Audited 38 sports trls--100% msn success/81M lbs worldwide
- Expertly mng&#39;d/tracked yearly FHP; accuracy critical to 1.5K sorties--240M lbs fuel offluaded/removed broken NVG test scores; leveraged residents to resolve installation activity
- Elected Vice President of ### RQW Tiger Team; managed scheduling for over 1,600 graduates and 270 students
- Squadron Rescue Team Response for MDG FY Univ foster cost to gov&#39;t-safe groups 330 general of servers--safeguarded $337K entitlements
- USCYBERCOM J641s; manifested/coord&#39;d $21M pax/flew out supply sys--enabled AF-wide GACFLILE 14
- Managed 6K ft&#39;s wing&#39;s 2nd biggest OIF; completed 200+ mx actions--13/13 asigned acft returned to flying 8 hr

In [31]:
print(generate_text(model, start_string=u"- Led 6 org test event; executed 250k runs--"))

- Led 6 org test event; executed 250k runs--secured $33M ntwk assets/instr&#39;d 4 civic certifiers
- Conducted 12 hours of Status of Excellence hrs--3.8 GPA
- SEL for J6 CAPs; ensured complete/accurate file/program--reduced errors to modem AF wide
- Force multiplier for 4 AF pilots; gteat contact w/ zero defects; spt&#39;d SQ&#39;s 92% QA pass rate--best seen in 2 yrs
- Replaced two faulty air conditioner alternators; skyrocketed unit Mx insps--unit rated &quot;Excellent&quot;
- Excelled as expediter, TSgt position; led 18 loader 603 Amn/1.5K arrivatory sys
- Attended emergency responses ISO the Combat Armed Forces Week crypto trivia training--aligned with increasing responsibility--Promote
- Flawlessly executed records Custodian program; reduced requirements for CCAF degree--maintained 4.0 GPA
- Finished 16 hrs s/w security tng; earned 4 dvlpment certs--ensured strict storage for $21M system
- Audited LOGMOD testing; raised inspection noted 19 hours to AFT; won VoI Partners noted--ex

In [34]:
import json
json.dumps(char2idx)


&#39;{&quot;\\t&quot;: 0, &quot;\\n&quot;: 1, &quot; &quot;: 2, &quot;!&quot;: 3, &quot;\\&quot;&quot;: 4, &quot;#&quot;: 5, &quot;$&quot;: 6, &quot;%&quot;: 7, &quot;&amp;&quot;: 8, &quot;\&#39;&quot;: 9, &quot;(&quot;: 10, &quot;)&quot;: 11, &quot;*&quot;: 12, &quot;+&quot;: 13, &quot;,&quot;: 14, &quot;-&quot;: 15, &quot;.&quot;: 16, &quot;/&quot;: 17, &quot;0&quot;: 18, &quot;1&quot;: 19, &quot;2&quot;: 20, &quot;3&quot;: 21, &quot;4&quot;: 22, &quot;5&quot;: 23, &quot;6&quot;: 24, &quot;7&quot;: 25, &quot;8&quot;: 26, &quot;9&quot;: 27, &quot;:&quot;: 28, &quot;;&quot;: 29, &quot;&lt;&quot;: 30, &quot;=&quot;: 31, &quot;&gt;&quot;: 32, &quot;?&quot;: 33, &quot;@&quot;: 34, &quot;A&quot;: 35, &quot;B&quot;: 36, &quot;C&quot;: 37, &quot;D&quot;: 38, &quot;E&quot;: 39, &quot;F&quot;: 40, &quot;G&quot;: 41, &quot;H&quot;: 42, &quot;I&quot;: 43, &quot;J&quot;: 44, &quot;K&quot;: 45, &quot;L&quot;: 46, &quot;M&quot;: 47, &quot;N&quot;: 48, &quot;O&quot;: 49, &quot;P&quot;: 50, &quot;Q&q

In [35]:
json.dumps(vocab)

&#39;[&quot;\\t&quot;, &quot;\\n&quot;, &quot; &quot;, &quot;!&quot;, &quot;\\&quot;&quot;, &quot;#&quot;, &quot;$&quot;, &quot;%&quot;, &quot;&amp;&quot;, &quot;\&#39;&quot;, &quot;(&quot;, &quot;)&quot;, &quot;*&quot;, &quot;+&quot;, &quot;,&quot;, &quot;-&quot;, &quot;.&quot;, &quot;/&quot;, &quot;0&quot;, &quot;1&quot;, &quot;2&quot;, &quot;3&quot;, &quot;4&quot;, &quot;5&quot;, &quot;6&quot;, &quot;7&quot;, &quot;8&quot;, &quot;9&quot;, &quot;:&quot;, &quot;;&quot;, &quot;&lt;&quot;, &quot;=&quot;, &quot;&gt;&quot;, &quot;?&quot;, &quot;@&quot;, &quot;A&quot;, &quot;B&quot;, &quot;C&quot;, &quot;D&quot;, &quot;E&quot;, &quot;F&quot;, &quot;G&quot;, &quot;H&quot;, &quot;I&quot;, &quot;J&quot;, &quot;K&quot;, &quot;L&quot;, &quot;M&quot;, &quot;N&quot;, &quot;O&quot;, &quot;P&quot;, &quot;Q&quot;, &quot;R&quot;, &quot;S&quot;, &quot;T&quot;, &quot;U&quot;, &quot;V&quot;, &quot;W&quot;, &quot;X&quot;, &quot;Y&quot;, &quot;Z&quot;, &quot;[&quot;, &quot;\\\\&quot;, &quot;]&quot;, &quot